## 1. LangChain 中的提示模板

提示模板(prompt template)是指生成提示的可重复的方式。它包含一个文本字符串（“模板”），可以接收来自最终用户的一组参数并生成提示。

在Langchain 中的通过提示模板类（PromptTemplate）来构建提示的语句，本文将详细介绍如何通过PromptTemplate实现基于特定信息的问题回答。

## 2. 案例体验
🔹 本案例需使用 P100 及以上规格运行，请确保运行规格一致，可按照下图切换规格。

![](https://modelarts-labs-bj4-v2.obs.cn-north-4.myhuaweicloud.com/case_zoo/chatglm3/image/1.png)

🔹 点击Run in ModelArts，将会进入到ModelArts CodeLab中，这时需要你登录华为云账号，如果没有账号，则需要注册一个，且要进行实名认证，参考[《ModelArts准备工作_简易版》](https://developer.huaweicloud.com/develop/aigallery/article/detail?id=4ce709d6-eb25-4fa4-b214-e2e5d6b7919c) 即可完成账号注册和实名认证。 登录之后，等待片刻，即可进入到CodeLab的运行环境

🔹 出现 Out Of Memory ，请检查是否为您的参数配置过高导致，修改参数配置，重启kernel或更换更高规格资源进行规避❗❗❗

### 2.1 下载ChatGLM3模型

In [1]:
import os
import moxing as mox

work_dir = '/home/ma-user/work'
obs_path = 'obs://dtse-models/tar-models/chatglm3-6b.tar'
ma_path = os.path.join(work_dir, 'chatglm3-6b.tar')
mox.file.copy(obs_path, ma_path)

mox.file.copy_parallel('obs://modelarts-labs-bj4-v2/course/ModelBox/frpc_linux_amd64', '/home/ma-user/work/frpc_linux_amd64')

INFO:root:Using MoXing-v2.1.0.5d9c87c8-5d9c87c8
INFO:root:Using OBS-Python-SDK-3.20.9.1


进入chatglm3目录，解压模型压缩包

In [2]:
os.chdir(work_dir)
!pwd
!tar -xvf chatglm3-6b.tar

/home/ma-user/work
chatglm3-6b/
chatglm3-6b/.gitattributes
chatglm3-6b/MODEL_LICENSE
chatglm3-6b/README.md
chatglm3-6b/config.json
chatglm3-6b/configuration_chatglm.py
chatglm3-6b/modeling_chatglm.py
chatglm3-6b/pytorch_model-00001-of-00007.bin
chatglm3-6b/pytorch_model-00002-of-00007.bin
chatglm3-6b/pytorch_model-00003-of-00007.bin
chatglm3-6b/pytorch_model-00004-of-00007.bin
chatglm3-6b/pytorch_model-00005-of-00007.bin
chatglm3-6b/pytorch_model-00006-of-00007.bin
chatglm3-6b/pytorch_model-00007-of-00007.bin
chatglm3-6b/pytorch_model.bin.index.json
chatglm3-6b/quantization.py
chatglm3-6b/tokenization_chatglm.py
chatglm3-6b/tokenizer.model
chatglm3-6b/tokenizer_config.json


### 2.2 配置环境

本案例依赖Python3.10.10及以上环境，因此我们首先创建虚拟环境：

In [4]:
!/home/ma-user/anaconda3/bin/conda create -n python-3.10.10 python=3.10.10 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.10.10/bin/pip install ipykernel

/home/ma-user/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /home/ma-user/anaconda3/envs/python-3.10.10

  added / updated specs:
    - python=3.10.10


The following NEW packages will be INSTALLED:

  _libgcc_mutex      anaconda/pkgs/main/linux-64::_libgcc_mutex-0.1-main
  _openmp_mutex      anaconda/pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu
  bzip2              anaconda/pkgs/main/linux-64::bzip2-1.0.8-h7b6447c_0
  ca-certificates    anaconda/pkgs/main/linux-64::ca-certificates-2023.12.12-h06a4308_0
  ld_impl_linux-64   anaconda/pkgs/main/linux-64::ld_impl_linux-64-2.38-h1181459_1
  libffi             anaconda/pkgs/main/linux-64::libffi-3.4.4-h6a678d5_0
  libgcc-ng

In [6]:
import json
import os

data = {
   "display_name": "python-3.10.10",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.10.10/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/anaconda3/envs/PyTorch-1.8/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.10.10/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.10.10/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

创建完成后，稍等片刻，或刷新页面，点击右上角kernel选择python-3.10.10

![](https://modelarts-labs-bj4-v2.obs.cn-north-4.myhuaweicloud.com/case_zoo/chatglm3/image/2.png)

### 2.3 安装依赖库

In [1]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install langchain==0.0.329 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
  Using cached http://repo.myhuaweicloud.com/repository/pypi/packages/5b/0b/e45d26ccd28568013523e04f325432ea88a442b4e3020b757cf4361f0120/transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached http://repo.myhuaweicloud.com/repository/pypi/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached http://repo.myhuaweicloud.com/repository/pypi/packages/7f/c4/adcbe9a696c135578cabcbdd7331332daad4d49b7c43688bc2d36b3a47d2/huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached http://repo.myhuaweicloud.com/repository/pypi/packages/71/3c/3b1981c6a1986adc9ee7db760c0c34ea5b14ac3da9ecfcf1ea2a4ec6c398/numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Using cached http://repo.myhuaweicloud.com/repository/pypi/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8cb42e0d285e/PyYAML-6.0.1-cp310-cp310

## 3. 代码开发

首先加载ChatGLM3模型，然后基于langchain的PromptTemplate类来实现提示模板定义，具体代码如下：

### 3.1 基于langchain加载ChatGLM3模型

In [2]:
import os
from typing import Dict, Union, Optional
from typing import List

from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoTokenizer

#新建ChatGLMService类
class ChatGLMService(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None

    def __init__(self):
        super().__init__()

    #定义llm模型类型
    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    #定义类调用的逻辑
    def _call(self,
              prompt: str,
              stop: Optional[List[str]] = None) -> str:
        response, history = self.model.chat(
            self.tokenizer,
            prompt,
            history=self.history,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        if stop is not None:
            response = enforce_stop_tokens(response, stop)#基于结束词将内容切断
            
        self.history = self.history + [history[-1]]#将问题和答案传到历史记录中
        return response
    
    #加载ChatGLM3-6B模型
    def load_model(self,
                   model_name_or_path: str = "chatglm3-6b"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path,
            trust_remote_code=True
        )
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()
        self.model = self.model.eval()


work_dir = '/home/ma-user/work'

llm_model_name = 'chatglm3-6b'
llm_model_path = os.path.join(work_dir, llm_model_name)

chatLLM = ChatGLMService()
chatLLM.load_model(model_name_or_path=llm_model_path)

/home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:07<00:00,  1.10s/it]


In [3]:
res = chatLLM('睡眠不好很头疼，有什么解决方案')
print(res)

睡眠不好会导致头疼，可以尝试以下方法缓解头疼：

1. 改善睡眠环境：保持卧室安静、舒适、黑暗。避免在睡前使用电子设备，因为蓝光可能会干扰你的睡眠。选择一款舒适的床垫和枕头，保证睡眠时身体得到充分支撑。

2. 规律作息：尽量保持每天相同的入睡和起床时间，帮助身体建立稳定的生物钟。

3. 放松身心：睡前进行一些放松活动，如深呼吸、瑜伽、阅读等，有助于减轻压力，促进入睡。

4. 合理膳食：避免晚餐过量，尽量不要在临睡前摄入咖啡因、酒精等刺激性食物。

5. 锻炼身体：适当的锻炼有助于改善睡眠质量，但避免在睡前进行剧烈运动。

6. 头疼缓解方法：如果头疼持续不缓，可以尝试按摩头部、颈部，或用热毛巾敷在额头上。

7. 避免过度用脑：白天避免过度思考问题，保持心情轻松，有助于晚上更好地入睡。

如果头疼症状持续严重，建议及时就医，以便找出潜在的健康问题。


### 3.2 提示模板定义

提示模板的定义可以帮助模型更好的理解你的问题，以下为几个例子：

#### 3.2.1 说明回答要求

In [5]:
from langchain import PromptTemplate

def get_prompt_template1():
    prompt_template1 = """\
请尽量简洁来回答用户的问题。
问题:
{question}
"""

    prompt = PromptTemplate(template=prompt_template1,input_variables=["question"])
    return prompt
                                             
prompt1 = get_prompt_template1()

question1 = '睡眠不好很头疼，有什么解决方案'
message1 = prompt1.format(question=question1)
print(message1)

res1 = chatLLM(message1)
print('答案:')
print(res1)
                                             

请尽量简洁来回答用户的问题。
问题:
睡眠不好很头疼，有什么解决方案

答案:
改善睡眠环境、规律作息、放松身心、合理膳食、锻炼身体、缓解头疼方法。如有持续严重的头疼，建议就医。


#### 3.2.2 基于已知信息回答

In [6]:
from langchain import PromptTemplate

def get_prompt_template2():
    prompt_template2 = """
基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
已知内容:
{context}
问题:
{question}
    """
    prompt2 = PromptTemplate(template=prompt_template2,
                            input_variables=["context", "question"])
    return prompt2

prompt2 = get_prompt_template2()
context2 = '''
    晚上睡不着可能会让你感到焦虑或不舒服,但以下是一些可以帮助你入睡的方法:
    1. 制定规律的睡眠时间表:保持规律的睡眠时间表可以帮助你建立健康的睡眠习惯,使你更容易入睡。尽量在每天的相同时间上床,并在同一时间起床。
    2. 创造一个舒适的睡眠环境:确保睡眠环境舒适,安静,黑暗且温度适宜。可以使用舒适的床上用品,并保持房间通风。
    3. 放松身心:在睡前做些放松的活动,例如泡个热水澡,听些轻柔的音乐,阅读一些有趣的书籍等,有助于缓解紧张和焦虑,使你更容易入睡。
    4. 避免饮用含有咖啡因的饮料:咖啡因是一种刺激性物质,会影响你的睡眠质量。尽量避免在睡前饮用含有咖啡因的饮料,例如咖啡,茶和可乐。
    5. 避免在床上做与睡眠无关的事情:在床上做些与睡眠无关的事情,例如看电影,玩游戏或工作等,可能会干扰你的睡眠。
    6. 尝试呼吸技巧:深呼吸是一种放松技巧,可以帮助你缓解紧张和焦虑,使你更容易入睡。试着慢慢吸气,保持几秒钟,然后缓慢呼气。
    如果这些方法无法帮助你入睡,你可以考虑咨询医生或睡眠专家,寻求进一步的建议。
'''
question2 = '睡眠不好很头疼，有什么解决方案'

message2 = prompt2.format(context=context2, question=question2)
print(message2)

res2 = chatLLM(message2)
print('答案:')
print(res2)


基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
已知内容:

    晚上睡不着可能会让你感到焦虑或不舒服,但以下是一些可以帮助你入睡的方法:
    1. 制定规律的睡眠时间表:保持规律的睡眠时间表可以帮助你建立健康的睡眠习惯,使你更容易入睡。尽量在每天的相同时间上床,并在同一时间起床。
    2. 创造一个舒适的睡眠环境:确保睡眠环境舒适,安静,黑暗且温度适宜。可以使用舒适的床上用品,并保持房间通风。
    3. 放松身心:在睡前做些放松的活动,例如泡个热水澡,听些轻柔的音乐,阅读一些有趣的书籍等,有助于缓解紧张和焦虑,使你更容易入睡。
    4. 避免饮用含有咖啡因的饮料:咖啡因是一种刺激性物质,会影响你的睡眠质量。尽量避免在睡前饮用含有咖啡因的饮料,例如咖啡,茶和可乐。
    5. 避免在床上做与睡眠无关的事情:在床上做些与睡眠无关的事情,例如看电影,玩游戏或工作等,可能会干扰你的睡眠。
    6. 尝试呼吸技巧:深呼吸是一种放松技巧,可以帮助你缓解紧张和焦虑,使你更容易入睡。试着慢慢吸气,保持几秒钟,然后缓慢呼气。
    如果这些方法无法帮助你入睡,你可以考虑咨询医生或睡眠专家,寻求进一步的建议。

问题:
睡眠不好很头疼，有什么解决方案
    
答案:
睡眠不好可能会导致头疼，以下是一些建议帮助缓解头疼并改善睡眠质量：

1. 改善睡眠环境：保持规律的睡眠时间表，创造一个舒适、安静、黑暗且温度适宜的睡眠环境。
2. 放松身心：睡前进行一些放松活动，如泡热水澡、听轻柔音乐、阅读有趣书籍等，以缓解紧张和焦虑。
3. 避免刺激性饮料：避免在睡前饮用含有咖啡因的饮料，如咖啡、茶和可乐。
4. 避免在床上做与睡眠无关的事情：在床上避免做与睡眠无关的事情，如看电影、玩游戏或工作等。
5. 尝试呼吸技巧：深呼吸可以帮助你放松紧张和焦虑，从而更容易入睡。

如果这些方法无法帮助你入睡，可以考虑咨询医生或睡眠专家，寻求进一步的建议。


### 作业1
基于[课程2](https://developer.huaweicloud.com/develop/aigallery/notebook/detail?id=2ebc801b-d828-42af-a6ec-5b3893993a7c)和课程3（本案例），实现一个基于ChatGLM3+LangChain的聊天应用，需要有Gradio界面。

In [1]:
!pip install transformers==4.30.2
!pip install sentencepiece==0.1.99
!pip install torch==2.0.1
!pip install gradio==3.39 mdtex2html -i https://pypi.tuna.tsinghua.edu.cn/simple --trusted-host pypi.tuna.tsinghua.edu.cn
!cp /home/ma-user/work/frpc_linux_amd64 /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /home/ma-user/anaconda3/envs/python-3.10.10/lib/python3.10/site-packages/gradio/frpc_linux_amd64_v0.2

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
import os
import gradio as gr
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoTokenizer

# 新建 ChatGLMService 类
class ChatGLMService(LLM):
    max_token: int = 10000
    temperature: float = 0.1
    top_p = 0.9
    history = []
    tokenizer: object = None
    model: object = None

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "ChatGLM"

    def _call(self, prompt: str, stop=None) -> str:
        response, history = self.model.chat(
            self.tokenizer,
            prompt,
            history=self.history,
            max_length=self.max_token,
            temperature=self.temperature,
        )
        if stop is not None:
            response = enforce_stop_tokens(response, stop)

        self.history = self.history + [history[-1]]
        return response

    def load_model(self, model_name_or_path: str = "chatglm3-6b"):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path,
            trust_remote_code=True
        )
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half().cuda()
        self.model = self.model.eval()

# 创建 ChatGLMService 实例并加载模型
work_dir = '/home/ma-user/work'
llm_model_name = 'chatglm3-6b'
llm_model_path = os.path.join(work_dir, llm_model_name)
chatLLM = ChatGLMService()
chatLLM.load_model(model_name_or_path=llm_model_path)

#去除输入框的内容
def reset_user_input():
    return gr.update(value='')

#清除状态
def reset_state():
    return [], [], None



# Gradio 界面
with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">ChatGLM3-6B</h1>""")

    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            with gr.Column(scale=12):
                user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=10, container=False)
            with gr.Column(min_width=32, scale=1):
                submitBtn = gr.Button("Submit", variant="primary")
        with gr.Column(scale=1):
            emptyBtn = gr.Button("Clear History")
            max_length = gr.Slider(0, 32768, value=8192, step=1.0, label="Maximum length", interactive=True)
            top_p = gr.Slider(0, 1, value=0.8, step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)

    history = gr.State([])
    past_key_values = gr.State(None)

    def predict(input, chatbot, max_length, top_p, temperature, history, past_key_values):
        chatbot.append((input, chatLLM(input, stop=["<eos>"])))
        for response, history, past_key_values in chatLLM.stream_chat(
            input, history, past_key_values=past_key_values,
            return_past_key_values=True,
            max_length=int(max_length), top_p=float(top_p),
            temperature=float(temperature)
        ):
            chatbot[-1] = (input, response)
            yield chatbot, history, past_key_values

    submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history, past_key_values],
                    [chatbot, history, past_key_values], show_progress=True)
    submitBtn.click(reset_user_input, [], [user_input])

    emptyBtn.click(reset_state, outputs=[chatbot, history, past_key_values], show_progress=True)

demo.queue().launch(share=True, inbrowser=True)


Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.25s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 15.90 GiB total capacity; 15.21 GiB already allocated; 11.75 MiB free; 15.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF